Import Carbonara's Data Tools by running the below cell (_ctrl+enter_)

In [1]:
import CarbonaraDataTools as CDT
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.io as pio
pio.renderers.default='notebook'

1. Enter the path to your protein (it should be inside newFitData)

In [2]:
# <-- Run this cell to enter a name for your prediction
molPath_text = widgets.Text(
    value='newFitData/DisulfideLinkagesExample/',
    placeholder='Enter the path to your protein:',
    description='Enter the path to your protein:',
    disabled=False,
    layout={'width': '550px','height':'50px'},   
)

molPath_text.style = {'description_width': '200px','description_height': '25px'}

def molPath_handler(change):
    global molPath
    molPath = change.new
    with output:
        clear_output(wait=True)

molPath = 'newFitData/DisulfideLinkagesExample/'
# Output widget
output = widgets.Output()
molPath_text.observe(molPath_handler,names='value')
display(molPath_text)

Text(value='newFitData/DisulfideLinkagesExample/', description='Enter the path to your protein:', layout=Layou…

2. Enter the name you gave to the specific run you'd like to analyse

In [3]:
# <-- Run this cell to enter a name for this run
runName_text = widgets.Text(
    value='fitdata',
    placeholder='Enter the name of the run you want to analyse:',
    description='Enter the name of the run you want to analyse:',
    disabled=False,
    layout={'width': '450px','height':'50px'},   
)

runName_text.style = {'description_width': '300px','description_height': '25px'}

def runName_handler(change):
    global run_name
    run_name = change.new
    with output:
        clear_output(wait=True)

run_name = 'fitdata'
# Output widget
output = widgets.Output()
runName_text.observe(runName_handler,names='value')
display(runName_text)

Text(value='fitdata', description='Enter the name of the run you want to analyse:', layout=Layout(height='50px…

3. Get the log files for this run

In [4]:
logs = CDT.getLogs(molPath,run_name)
logs

['newFitData/DisulfideLinkagesExample/fitdata/fitLog1.dat']

4. Check the progress of your run by selecting a log file from the dropdown

In [5]:
# <-- Run this cell to view the dropdown box
logFile_Dropdown = widgets.Dropdown(
    options = logs,
    value = logs[0],
    description = 'Select a logFile from the dropdown box ',
    disabled=False,
    layout={'width': '550px','height':'25px'}
)

# Function to display tail of DataFrame
def display_tail(log_path):
    df = CDT.LogFile2df(log_path)
    display(df.tail())

def dropdown_handler(change):
    global log_path
    log_path = change.new
    with output:
        clear_output(wait=True)  # Clear previous output
        display_tail(log_path)
        CDT.viewBestSAXSFit(molPath,log_path).show()
        CDT.viewBestMolChange(molPath,log_path)

# Apply CSS style to the dropdown box to make it more visible
logFile_Dropdown.style = {'description_width': '250px','description_height': '25px'}

# Output widget
output = widgets.Output()

# Display initial output
log_path = logs[0]
with output:
    display_tail(log_path)
    CDT.viewBestSAXSFit(molPath,log_path).show()
    CDT.viewBestMolChange(molPath,log_path)

logFile_Dropdown.observe(dropdown_handler,names='value')
display(logFile_Dropdown)
display(output)

Dropdown(description='Select a logFile from the dropdown box ', layout=Layout(height='25px', width='550px'), o…

Output()

5. View the changes to your structure as we inlcuded more of the SAXs data.

In [6]:
# <-- Run this cell to view the q slider

q = CDT.getqChanges(log_path)
qrange_slider = widgets.SelectionSlider(
    options=q,
    value = q[0],
    description='Max q:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout={'width': '400px', 'height': '100px'}
    )

def slider_handler(change):
    global q_choice
    q_choice = change.new
    with output:
        clear_output(wait=False)
        saxs_fl, mol_fl = CDT.getSAXsandMolFile(log_path,q_choice)
        CDT.plotMolAndSAXS(MolPath,saxs_fl,mol_fl).show("notebook")

display(qrange_slider)
saxs_fl, mol_fl = CDT.getSAXsandMolFile(log_path,q[0])
# Output widget
output = widgets.Output()
with output:
    CDT.plotMolAndSAXS(MolPath,saxs_fl,mol_fl).show("notebook")   
qrange_slider.observe(slider_handler,names='value')
display(output)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
CDT.PlotBestOutputRMSD(MolPath,RunName)

In [ ]:
CDT.cluster(CDT.BestOutputRMSD(MolPath,RunName))

In [ ]:
CDT.plotBestSKMTComp(MolPath,RunName)

In [ ]:
CDT.overlayCluster(MolPath,RunName,0)